In [ ]:
# Pinggy script
def start_pinggy_tunnel(args = ""):
    # Saving from Github Wanda Web
    !wget https://raw.githubusercontent.com/wandaweb/jupyter-webui-tunneling/main/pinggy.py -O {working_dir}/pinggy.py
    # For Running ComfyUI
    %cd {working_dir}

    # Running Command
    !python pinggy.py --command="{python} {working_dir}/ComfyUI/main.py {args}" --port=8188

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request
from tqdm.auto import tqdm

def iframe_thread(port):
    # Waiting for ComfyUI to start at port
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    # Comfy UI has started
    print("\n")
    print("ComfyUI finished loading! Launching Local Tunnel\n")
    print("if it gets stuck here, then Local Tunnel is having issues)\n")

    # This is to get the IP Address to access local tunnel
    kernel_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    # Starting Local Tunnel
    lt_process = subprocess.Popen(["lt", "--port", f"{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print("LocalTunnel Password:",kernel_ip)
    # Printing out Local Tunnel Processes (including the local tunnel ip address)
    for line in lt_process.stdout:
        print(line, end='')


# Use this to start local tunnel
def start_local_tunnel():
    # Try and install local tunnel
    !npm install -g localtunnel

    # Start the main.py process
    main_process = run_comfy_ui_background()

    # Start the iframe thread
    iframe_thread_obj = threading.Thread(target=iframe_thread, daemon=True, args=(8188,))
    iframe_thread_obj.start()

    print("Starting ComfyUI Local Tunnel Background Process")

    # Create a thread to wait for the iframe thread to complete
    def wait_for_iframe():
        iframe_thread_obj.join()

    threading.Thread(target=wait_for_iframe, daemon=True).start()